In [1]:
#!pip install isoweek
#!pip install pyarrow

In [2]:
import pandas as pd
import numpy as np
import datetime
# pip install isoweek
from isoweek import Week
from pandas_summary import DataFrameSummary
from datetime import datetime

In [3]:
PATH_PROCESSED='processed/'
PATH = 'dataset/'

user, occupation = [pd.read_csv(PATH + fname+'.csv', low_memory=False, sep="|") for fname in ['user', 'occupation']]
info, train = [pd.read_csv(PATH + fname+'.csv', low_memory=False, sep="\t") for fname in ['info', 'train']]
test = pd.read_csv(PATH + 'test.csv', low_memory=False, sep=",")
item = pd.read_csv(PATH + 'item.csv',  low_memory=False, encoding='latin-1', sep="|")

item.rename( columns={'movieId' :'movie_id', 'movieTitle' :'movie_title', 'release date' :'release_date', 'video release date' :'video_release_date', 'IMDb URL' :'imdb_url', 'unknown' :'unknown', 'Action' :'action', 'Adventure' :'adventure', 'Animation' :'animation', 'Children\'s' :'children', 'Comedy' :'comedy', 'Crime' :'crime', 'Documentary' :'documentary', 'Drama' :'drama', 'Fantasy' :'fantasy', 'Film-Noir' :'film-noir', 'Horror' :'horror', 'Musical' :'musical', 'Mystery' :'mystery', 'Romance' :'romance', 'Sci-Fi' :'sci-fi', 'Thriller' :'thriller', 'War' :'war', 'Western' :'western'}, inplace=True)

user.rename( columns={'user id' :'user_id', 'age' :'age', 'gender' :'gender', 'occupation' :'occupation', 'zip code' :'zip_code'}, inplace=True)

train.rename( columns={'userId' :'user_id', 'movieId' :'movie_id'}, inplace=True)
test.rename( columns={'Id' :'id', 'userId' :'user_id', 'movieId' :'movie_id'}, inplace=True)
#data = train + test
data = train
ratings = data

u_unique = data.user_id.unique()
user2Idx = {o:i+1 for i,o in enumerate(u_unique)}

m_unique = data.movie_id.unique()
movie2Idx = {o:i+1 for i,o in enumerate(m_unique)}

In [4]:
table_names = ['user', 'item']
dfs = [user, item]

In [5]:
# Max timestamp = 893286638
print(max(data.timestamp))
date_today=max(data.timestamp)
timestamp_today = datetime.fromtimestamp(date_today)

893286638


# Missing data

In [6]:
def get_missing_columns(df):
    return list(df.columns[df.describe(include = 'all').loc['count']<len(df)])

In [7]:
for i, df in enumerate(dfs):
    print(table_names[i], get_missing_columns(df))

user []
item ['release_date', 'video_release_date', 'imdb_url']


In [8]:
# Create a NEW user without votes
if len(user[user['user_id'] == 944]) < 1:
    theNewUser = {
        'user_id': [944, 945],
        'age': [24, 28],
        'gender': ['M', 'F'],
        'occupation': ['technician', 'writer'],
        'zip_code': [85711, 94043]
    }

    newUsers = pd.DataFrame(theNewUser)
    newUsers.to_csv(PATH + 'u.user', mode='a', index=False, header=False, sep="|")

In [9]:
item.loc[item['release_date'].isna() | (len(item['release_date'])<2), 'release_date'] = np.int32(1990)
display(DataFrameSummary(item).summary())

,movie_id,movie_title,release_date,video_release_date,imdb_url,unknown,action,adventure,animation,children,...,fantasy,film-noir,horror,musical,mystery,romance,sci-fi,thriller,war,western
count,1682.0,NaN,NaN,0.0,NaN,1682.0,1682.0,1682.0,1682.0,1682.0,...,1682.0,1682.0,1682.0,1682.0,1682.0,1682.0,1682.0,1682.0,1682.0,1682.0
mean,841.5,NaN,NaN,NaN,NaN,0.001189,0.149227,0.080262,0.02497,0.072533,...,0.01308,0.014269,0.054697,0.033294,0.036266,0.146849,0.060048,0.149227,0.042212,0.016052
std,485.695893,NaN,NaN,NaN,NaN,0.034473,0.356418,0.271779,0.156081,0.259445,...,0.11365,0.118632,0.227455,0.179456,0.187008,0.354061,0.237646,0.356418,0.201131,0.125714
min,1.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25%,421.25,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50%,841.5,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
75%,1261.75,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
max,1682.0,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
counts,1682,1682,1682,0,1679,1682,1682,1682,1682,1682,...,1682,1682,1682,1682,1682,1682,1682,1682,1682,1682
uniques,1682,1664,241,0,1660,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2


In [10]:
for i, df in enumerate(dfs):
    print(table_names[i], get_missing_columns(df))

user []
item ['video_release_date', 'imdb_url']


# Paso a fecha algunas columnas para despues poder ponerla en días respecto a fecha

In [11]:
item['release_date'] = item['release_date'].apply(lambda x: str(x))

# Función para hacer el join

In [12]:
def join_df(left, right, left_on, right_on=None):
    if right_on is None: right_on = left_on
    return left.merge(right, how='left', left_on=left_on, right_on=right_on, 
                      suffixes=("", "_y"))

In [13]:
trainItem = join_df(data, item, "movie_id", "movie_id")
joined_train = join_df(trainItem, user, "user_id", "user_id")
joined_train

,user_id,movie_id,rating,timestamp,movie_title,release_date,video_release_date,imdb_url,unknown,action,...,mystery,romance,sci-fi,thriller,war,western,age,gender,occupation,zip_code
0,1,1,5,874965758,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,...,0,0,0,0,0,0,24,M,technician,85711
1,1,2,3,876893171,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,...,0,0,0,1,0,0,24,M,technician,85711
2,1,3,4,878542960,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,...,0,0,0,1,0,0,24,M,technician,85711
3,1,4,3,876893119,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,...,0,0,0,0,0,0,24,M,technician,85711
4,1,5,3,889751712,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,...,0,0,0,1,0,0,24,M,technician,85711
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79995,943,1067,2,875501756,Bottle Rocket (1996),21-Feb-1996,NaN,http://us.imdb.com/M/title-exact?Bottle%20Rock...,0,0,...,0,0,0,0,0,0,22,M,student,77841
79996,943,1074,4,888640250,Reality Bites (1994),01-Jan-1994,NaN,http://us.imdb.com/M/title-exact?Reality%20Bit...,0,0,...,0,0,0,0,0,0,22,M,student,77841
79997,943,1188,3,888640250,Young Guns II (1990),01-Jan-1990,NaN,http://us.imdb.com/M/title-exact?Young%20Guns%...,0,1,...,0,0,0,0,0,1,22,M,student,77841
79998,943,1228,3,888640275,Under Siege 2: Dark Territory (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Under%20Siege...,0,1,...,0,0,0,0,0,0,22,M,student,77841


In [14]:
joined_train.loc[joined_train['timestamp'].isna() | (len(joined_train['timestamp'])<2), 'timestamp'] = date_today

joined_train.loc[joined_train['release_date'].isna() | (len(joined_train['release_date'])<2), 'release_date'] = np.int32(1990)

In [15]:
%time joined_train.to_feather(PATH_PROCESSED + 'joined_train.fth')

CPU times: user 155 ms, sys: 36.6 ms, total: 192 ms
Wall time: 202 ms


# Creando archivo con k-movies mejor rankeadas

In [16]:
movie_ratings = data.groupby("movie_id").mean()
movie_ratings.drop('user_id', inplace=True, axis=1)
movie_ratings.drop('timestamp', inplace=True, axis=1)
movie_ratings.rating.sort_index()
sorted_ratings = movie_ratings.rating.sort_values(ascending=False)

In [17]:
k = 20
save_ratings = sorted_ratings[:k]
save_ratings.to_csv(PATH_PROCESSED + 'top_k_movies.csv')
save_ratings

movie_id
1189    5.000000
1653    5.000000
1599    5.000000
1500    5.000000
1467    5.000000
1201    5.000000
1122    5.000000
1293    5.000000
1449    4.714286
1367    4.666667
408     4.559140
169     4.525773
850     4.500000
1594    4.500000
1642    4.500000
318     4.489451
483     4.441026
64      4.417040
12      4.398104
50      4.359504
Name: rating, dtype: float64

In [18]:
# datetime object containing current date and time
now = datetime.now()
# dd/mm/YY H:M:S
dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
print("date and time =", dt_string)

date and time = 31/12/2022 09:42:56
